In [1]:
from IPython.display import display
from pathlib import Path
from ff_epl.optimisation import optimise

# The problem: select a set of players for a fantasy football team such that we maximise the number of points based on an adjusted historical average.

Let $x$ be a vector of players to be selected - our decision variables - with each player indexed by an identification number within set $P$. Let four boolean vectors $g$, $d$, $m$ and $f$ define whether each players is a goalkeeper, defender, midfielder and forward, respectively. Let $c$ and $w$ be the cost and optimisation weight of each player.

Although we're selecting the 11 outfield players we still need to adjust the our budget to ensure we can select bench players; for now, let's assume we can take the cheapest players for each position to sit on the bench. As such, introduce four slack variables to capture the amount of bench players we need for each position; $B^{g}$, $B^{d}$, $B^{m}$, and $B^{f}$. We also need the minimum cost of players for each of these positions, $C^{g}$, $C^{d}$, $C^{m}$, and $C^{f}$. Let $q$ contain the set of position indices, $Q=\{g,d,m,f\}$.

We cannot select more than three players from each team; we will use a boolean vector, $t$, for each team, $t*$, in the EPL where $t \in EPL$ (e.g. $T^{ARS}$ ).

We also don't want to select any players that are injured at the start of the season.

All contraints noted above are outlined inline below.

# Model

$maximise$

$w^{T} x$

$subject \ to$

$\sum\limits_{p=0}^{P}x_{p} = 11$  (select 11 outfield players)

$\sum\limits_{p=0}^{P}x_{p}g_{p} = 1$  (select 1 goalkeeper)

$\sum\limits_{p=0}^{P}x_{p}d_{p} \geq 3$, $\sum\limits_{p=0}^{P}x_{p}d_{p} \leq 5$   (select three to five defenders)

$\sum\limits_{p=0}^{P}x_{p}m_{p} \leq 5$  (select no more than five midfielders)

$\sum\limits_{p=0}^{P}x_{p}f_{p} \geq 1$  (select at least one forward)


$\sum\limits_{p=0}^{P}x_{p}g_{p} + B^{g} = 2$  (capture amount of bench goalkeepers required)

$\sum\limits_{p=0}^{P}x_{p}d_{p} + B^{d} = 5$ (capture amount of bench defenders required)

$\sum\limits_{p=0}^{P}x_{p}m_{p} + B^{m} = 5$ (capture amount of bench midfielders required)

$\sum\limits_{p=0}^{P}x_{p}f_{p} + B^{f} = 3$ (capture amount of bench forwards required)

$\sum\limits_{p=0}^{P}x_{p}c_{p} + \sum\limits_{q=0}^{Q} B^q C^q \leq 100$ (total cost of the team including bench must be less than 100)

$\sum\limits_{p=0}^{P}x_{p}t_{p}^{t*} \leq 3 \quad \forall t* \in EPL$ (select no more than three players from each club)

In [2]:
df = optimise.fetch_data(Path("data/player_points.csv"))  # Output from notebooks/metric_research.ipynb
decision_vars, mdl = optimise.generate_model(df, optimisation_metric="points_bias_recent_avg")
optimise.lpsolve(model=mdl, logs=True)

mdl.writeLP("FantasyFootball.lp")

# Get solution vector
SV = [bool(int(x.value())) for x in decision_vars]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ehutchinson/projects/fantasy-football-epl/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3aa322f737be4316ac764ac5bcdbb982-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/3aa322f737be4316ac764ac5bcdbb982-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 35 COLUMNS
At line 2933 RHS
At line 2964 BOUNDS
At line 3316 ENDATA
Problem MODEL has 30 rows, 351 columns and 1866 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1799.8 - 0.00 seconds
Cgl0004I processed model has 23 rows, 335 columns (335 integer (335 of which binary)) and 1268 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.666667
Cbc0038I Solution found of -1773.57
Cbc0038I Before mini branch and bound, 333 integers at bound fixed and 0 continuous
Cbc0038I Full problem 2

In [3]:
print(f'Team cost: ${df[SV]["now_cost"].sum(): .2f}')
display(df.loc[SV, ["id", "position", "name", "first_name", "second_name", "now_cost"]])

Team cost: $ 82.50


,id,position,name,first_name,second_name,now_cost
31,569,GKP,Wolves,José,Malheiro de Sá,5.0
53,5,DEF,Arsenal,Gabriel,dos Santos Magalhães,5.0
87,189,DEF,Chelsea,César,Azpilicueta,4.5
105,290,DEF,Liverpool,Trent,Alexander-Arnold,8.0
110,307,DEF,Liverpool,Andrew,Robertson,6.5
168,373,MID,Man Utd,Bruno,Borges Fernandes,8.5
182,14,MID,Arsenal,Martin,Ødegaard,8.5
211,516,MID,Spurs,Son,Heung-min,9.0
229,216,MID,Chelsea,Raheem,Sterling,7.0
247,308,MID,Liverpool,Mohamed,Salah,12.5
